In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('env')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [4]:
from langchain_google_genai.llms import GoogleGenerativeAI

c:\Users\habob\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm = GoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model='gemini-pro')

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
df = pd.read_csv('Data.csv')

## LLMChain

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [19]:
prompt = ChatPromptTemplate.from_template(
    "What is the best description to this  \
    {product}?"
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

In [26]:
product = "100004"
chain.run(product)

"**Descriptive Names:**\n\n* **Quadruple Infinity** (emphasizing the large number of products)\n* **Ten Thousand Plus** (highlighting the exact number)\n* **Limitless Innovations** (conveying a sense of endless possibilities)\n* **Abundant Creations** (implying a vast selection)\n* **Myriad Solutions** (suggesting a wide range of products)\n\n**Creative and Memorable Names:**\n\n* **Zenith** (representing the highest point or number)\n* **Everest** (evoking a sense of grandeur and magnitude)\n* **Odyssey** (suggesting a long and epic journey to create such a large product line)\n* **Mosaic** (referencing the diverse and interconnected nature of the products)\n* **Kaleidoscope** (capturing the vibrant and ever-changing array of offerings)\n\n**Names that Reflect the Company's Values:**\n\n* **Integrity Unlimited** (emphasizing the company's commitment to quality and honesty)\n* **Precision and Plenty** (highlighting the company's focus on accuracy and abundance)\n* **Innovation and Abun

## SimpleSequentialChain

In [21]:
from langchain.chains import SimpleSequentialChain

In [22]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [23]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [24]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [25]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
* **NumeriCorp**
* **TenK Squared**
* **Thousand Plus Two**
* **Precise Precision**
* **Exactus**
* **Quantify**
* **Accurate Analytics**
* **Precise Insights**
* **Data Precision**
* **Quantified Solutions**
NumeriCorp, a leader in data analytics, provides precise and accurate insights to help businesses quantify their success.

> Finished chain.


'NumeriCorp, a leader in data analytics, provides precise and accurate insights to help businesses quantify their success.'

## SequentialChain

In [27]:
from langchain.chains import SequentialChain

In [28]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [30]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [31]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [32]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["product_description"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [34]:
review = df.product_description[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Achieving delicious results is almost effortless with this Whirlpool over-the-range microwave hood with convection cooking. With 1.9 cu. ft. capacity, you'll have room to prepare multiple dishes at once. And, you won't have to micro-manage the cooking process thanks to the sensor cooking options. Sensor cooking tracks cooking progress and automatically adjusts cooking time. Convection roasting and baking allows you to bake cakes, bread, cookies and roast meats with ease. Clean-up is simple too with the industry leading CleanRelease non-stick interior surface. It easily releases cooked-on food without the use of harsh chemicals. A damp cloth or sponge is all that is needed to remove cooked-on spills and splashes.California residents: see&nbsp;Proposition 65 informationSpacious 1.9 cu. ft. capacity accommodates dinner plates and casserole dishes with ease1100 watts of cooking power and 10 cooking levels make cooking and reheating a snap400 CFM venting system whisks smoke, ste

## Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [38]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [39]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [40]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [43]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Blackbody radiation is the electromagnetic radiation within or surrounding a body in thermodynamic equilibrium with its environment, or emitted by a black body (an idealized opaque, non-reflective body). It has a specific spectrum of wavelengths that depends only on the body's temperature, which characterizes its thermal radiation. This radiation results from the thermal motion of charged particles within the body, and is a characteristic of all matter with a temperature greater than absolute zero."

In [44]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'4'